In [ ]:
import numpy as np
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
plt.style.use(r'PaperDoubleFig.mplstyle')

# FODO elements

In [ ]:
def focus(k, l):
    """
    This function could be used to generate "natural focusing" and Quand focusing(defocusing)
    """
    if k>0:
    
        sr_k = np.sqrt(k)
        c = np.cos(sr_k*l)
        s = np.sin(sr_k*l)
        return np.array([[c, s/sr_k],
                         [-sr_k*s, c]])
    elif k<0:
        sr_k = np.sqrt(-k)
        c = np.cosh(sr_k*l)
        s = np.sinh(sr_k*l)
        return np.array([[c, s/sr_k],
                         [sr_k*s, c]])

def drift(L):
    return np.array([[1, L],
                     [0, 1]])


# Genesis Parametrs
http://genesis.web.psi.ch/Manual/parameter_focusing.html

http://genesis.web.psi.ch/Manual/parameter_undulator.html

http://genesis.web.psi.ch/Manual/parameter_beam.html

In [ ]:
# UNDULATOR parameters
XLAMD = 0.0186  # undulator wavelength
ku = 2*np.pi/XLAMD
AW0 = 0.86  # rms Undulator parameter
K = np.sqrt(2)*AW0
# FODO parameters
F1ST = 5
QUADF = 30
FL = 10
QUADD = 30
FD = 10

DRL = 100
# Beam parameters
GAMMA0 = 12e9/0.511e6  # beam energy in mc2

# Derived parameters

In [ ]:
# Focusing quads
lf1st = F1ST*XLAMD
lf = FL*XLAMD
kf = 585*QUADF/GAMMA0
# Defocusing quads
ld = FD*XLAMD
kd = -585*QUADD/GAMMA0
# Undulator focusing
L = DRL*XLAMD
Ku = K**2*ku**2/(2*GAMMA0**2)

In [ ]:
# Thin lens approximation
f = (lf*kf)**-1
phi_c = 2*np.arcsin((lf+ld+2*L)/(4*f))
print(f"Simple phi_c={phi_c}")
print(f"Simple beta_max={(lf+ld+2*L)*(1+np.sin(phi_c/2))/np.sin(phi_c)}")
print(f"Simple beta_min={(lf+ld+2*L)*(1-np.sin(phi_c/2))/np.sin(phi_c)}")

# Building FODOs 

## Horizontal FODO

In [ ]:
FODOx = focus(kf, lf1st) @ \
        drift(L) @ focus(kd, ld) @ drift(L) @ \
        focus(kf, lf1st)
print(FODOx)

In [ ]:
phi_c = (np.arccos(np.trace(FODOx)/2))
print(phi_c)

In [ ]:
beta_x = FODOx[0,1]/np.sin(phi_c)
print(beta_x)

In [ ]:
np.sqrt(beta_x*0.2e-6/GAMMA0)

## Vertical FODO without "natural focusing"

In [ ]:
FODOy = focus(-kf, lf1st) @ \
        drift(L) @ focus(-kd, ld) @ drift(L) @ \
        focus(-kf, lf1st)
print(FODOy)

In [ ]:
phi_c = (np.arccos(np.trace(FODOy)/2))
print(phi_c)

In [ ]:
beta_y = FODOy[0,1]/np.sin(phi_c)
print(beta_y)

In [ ]:
np.sqrt(beta_y*0.2e-6/GAMMA0)

## Vertical FODO with "natural focusing"

In [ ]:
FODOy = focus(-kf, lf1st) @ \
        focus(Ku, L) @ focus(-kd, ld) @ focus(Ku, L) @ \
        focus(-kf, lf1st)
print(FODOy)

In [ ]:
phi_c = (np.arccos(np.trace(FODOy)/2))
print(phi_c)

In [ ]:
beta_y = FODOy[0,1]/np.sin(phi_c)
print(beta_y)

In [ ]:
np.sqrt(beta_y*0.2e-6/GAMMA0)

# Values at Half lattice

In [ ]:
M1 = focus(kd, ld/2) @ drift(L) @ focus(kf, lf1st)
M2 = focus(kf, lf1st) @ drift(L) @ focus(kd, ld/2)
FODOx = M2@M1
phi_c = (np.arccos(np.trace(FODOx)/2))
print(phi_c)
beta_x = FODOx[0,1]/np.sin(phi_c)
print(np.sqrt(beta_x*0.2e-6/GAMMA0))
# reverse order
FODOx = M1@M2
phi_c = (np.arccos(np.trace(FODOx)/2))
print(phi_c)
beta_x = FODOx[0,1]/np.sin(phi_c)
print(np.sqrt(beta_x*0.2e-6/GAMMA0))
print('phase advance is the same!')

In [ ]:
M1 = focus(-kd, ld/2) @ focus(Ku, L) @ focus(-kf, lf1st)
M2 = focus(-kf, lf1st) @ focus(Ku, L) @ focus(-kd, ld/2)
FODOy = M2 @ M1
phi_c = (np.arccos(np.trace(FODOy)/2))
print(phi_c)
beta_y = FODOy[0,1]/np.sin(phi_c)
print(np.sqrt(beta_y*0.2e-6/GAMMA0))
# reverse order
FODOy = M1 @ M2
phi_c = (np.arccos(np.trace(FODOy)/2))
print(phi_c)
beta_y = FODOy[0,1]/np.sin(phi_c)
print(np.sqrt(beta_y*0.2e-6/GAMMA0))

We see that values at Half lattice point are not equal, which means that the beam is elliptic.
Let us adjust value of the quads in order to keep beam round on average

# FODO lattice tuning

In [ ]:
def cost(kf, kd):
    M1 = focus(kd, ld/2) @ drift(L) @ focus(kf, lf1st)
    M2 = focus(kf, lf1st) @ drift(L) @ focus(kd, ld/2)
    FODOx = M2@M1
    phi_c = (np.arccos(np.trace(FODOx)/2))
    beta_x_max = FODOx[0,1]/np.sin(phi_c)
    FODOx = M1@M2
    phi_c_x = (np.arccos(np.trace(FODOx)/2))
    beta_x_min = FODOx[0,1]/np.sin(phi_c_x)

    M1 = focus(-kd, ld/2) @ focus(Ku, L) @ focus(-kf, lf1st)
    M2 = focus(-kf, lf1st) @ focus(Ku, L) @ focus(-kd, ld/2)
    FODOy = M2 @ M1
    phi_c = (np.arccos(np.trace(FODOy)/2))
    beta_y_min = FODOy[0,1]/np.sin(phi_c)
    FODOy = M1 @ M2
    phi_c_y = (np.arccos(np.trace(FODOy)/2))
    beta_y_max = FODOy[0,1]/np.sin(phi_c_y)
    #return (phi_c_x-phi_c_y)**2
    return (beta_x_max-beta_y_max)**2 + (beta_x_min-beta_y_min)**2

npcost = np.vectorize(cost)

In [ ]:
xkf = np.linspace(0.99*kf, 1.01*kf)
ykd = np.linspace(0.99*kd, 1.01*kd)
X, Y = np.meshgrid(xkf, ykd)
res = npcost(X, Y)
plt.contourf(X, Y, res)
plt.colorbar()
plt.show()

In [ ]:
from scipy.optimize import minimize_scalar

In [ ]:
kf = minimize_scalar(cost, bracket=(0.95*kf, kf, 1.05*kf), args=kd).x

In [ ]:
M1 = focus(kd, ld/2) @ drift(L) @ focus(kf, lf1st)
M2 = focus(kf, lf1st) @ drift(L) @ focus(kd, ld/2)
FODOx = M2@M1
phi_c = (np.arccos(np.trace(FODOx)/2))
print(phi_c)
beta_x = FODOx[0,1]/np.sin(phi_c)
print(np.sqrt(beta_x*0.2e-6/GAMMA0))
# reverse order
FODOx = M1@M2
phi_c = (np.arccos(np.trace(FODOx)/2))
print(phi_c)
beta_x = FODOx[0,1]/np.sin(phi_c)
print(np.sqrt(beta_x*0.2e-6/GAMMA0))
print('phase advance is the same!')

In [ ]:
M1 = focus(-kd, ld/2) @ focus(Ku, L) @ focus(-kf, lf1st)
M2 = focus(-kf, lf1st) @ focus(Ku, L) @ focus(-kd, ld/2)
FODOy = M2 @ M1
phi_c = (np.arccos(np.trace(FODOy)/2))
print(phi_c)
beta_y = FODOy[0,1]/np.sin(phi_c)
print(np.sqrt(beta_y*0.2e-6/GAMMA0))
# reverse order
FODOy = M1 @ M2
phi_c = (np.arccos(np.trace(FODOy)/2))
print(phi_c)
beta_y = FODOy[0,1]/np.sin(phi_c)
print(np.sqrt(beta_y*0.2e-6/GAMMA0))

- We can see that phase advances are different although beam is matched;


In [ ]:
print(f"QUADF={GAMMA0*kf/585}")
print(f"QUADD={GAMMA0*kd/585}")

# FODO lattice tuning by phase

In [ ]:
def cost(kf, kd):
    M1 = focus(kd, ld/2) @ drift(L) @ focus(kf, lf1st)
    M2 = focus(kf, lf1st) @ drift(L) @ focus(kd, ld/2)
    FODOx = M2@M1
    phi_c = (np.arccos(np.trace(FODOx)/2))
    beta_x_max = FODOx[0,1]/np.sin(phi_c)
    FODOx = M1@M2
    phi_c_x = (np.arccos(np.trace(FODOx)/2))
    beta_x_min = FODOx[0,1]/np.sin(phi_c_x)

    M1 = focus(-kd, ld/2) @ focus(Ku, L) @ focus(-kf, lf1st)
    M2 = focus(-kf, lf1st) @ focus(Ku, L) @ focus(-kd, ld/2)
    FODOy = M2 @ M1
    phi_c = (np.arccos(np.trace(FODOy)/2))
    beta_y_min = FODOy[0,1]/np.sin(phi_c)
    FODOy = M1 @ M2
    phi_c_y = (np.arccos(np.trace(FODOy)/2))
    beta_y_max = FODOy[0,1]/np.sin(phi_c_y)
    return (phi_c_x-phi_c_y)**2
    #return (beta_x_max-beta_y_max)**2 + (beta_x_min-beta_y_min)**2

npcost = np.vectorize(cost)

In [ ]:
xkf = np.linspace(0.99*kf, 1.01*kf)
ykd = np.linspace(0.99*kd, 1.01*kd)
X, Y = np.meshgrid(xkf, ykd)
res = npcost(X, Y)
plt.contourf(X, Y, res)
plt.colorbar()
plt.show()

In [ ]:
kf = minimize_scalar(cost, bracket=(0.95*kf, kf, 1.05*kf), args=kd).x

In [ ]:
M1 = focus(kd, ld/2) @ drift(L) @ focus(kf, lf1st)
M2 = focus(kf, lf1st) @ drift(L) @ focus(kd, ld/2)
FODOx = M2@M1
phi_c = (np.arccos(np.trace(FODOx)/2))
print(phi_c)
beta_x = FODOx[0,1]/np.sin(phi_c)
print(np.sqrt(beta_x*0.2e-6/GAMMA0))
# reverse order
FODOx = M1@M2
phi_c = (np.arccos(np.trace(FODOx)/2))
print(phi_c)
beta_x = FODOx[0,1]/np.sin(phi_c)
print(np.sqrt(beta_x*0.2e-6/GAMMA0))
print('phase advance is the same!')

In [ ]:
M1 = focus(-kd, ld/2) @ focus(Ku, L) @ focus(-kf, lf1st)
M2 = focus(-kf, lf1st) @ focus(Ku, L) @ focus(-kd, ld/2)
FODOy = M2 @ M1
phi_c = (np.arccos(np.trace(FODOy)/2))
print(phi_c)
beta_y = FODOy[0,1]/np.sin(phi_c)
print(np.sqrt(beta_y*0.2e-6/GAMMA0))
# reverse order
FODOy = M1 @ M2
phi_c = (np.arccos(np.trace(FODOy)/2))
print(phi_c)
beta_y = FODOy[0,1]/np.sin(phi_c)
print(np.sqrt(beta_y*0.2e-6/GAMMA0))

In [ ]:
print(f"QUADF={GAMMA0*kf/585}")
print(f"QUADD={GAMMA0*kd/585}")